In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import shapiro, kstest, probplot
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

## Data Load

In [ ]:
# open dataset
df_pontuation = pd.read_csv('./datasets/pontuation.csv')

## EDA

In [ ]:
# Statistics analysis of the variables
df_pontuation.describe()

In [ ]:
# dispersion plt
# X = study_hours
# y = pontuation
sns.scatterplot(data=df_pontuation, x='study_hours', y='pontuation')

In [ ]:
# verify outliers in independent variable
sns.boxplot(df_pontuation, y='study_hours')

In [ ]:
# verify outliers in dependent variable
sns.boxplot(df_pontuation, y='pontuation')

In [ ]:
# Verify correlation - Pearson
sns.heatmap(df_pontuation.corr('pearson'), annot=True)

In [ ]:
# Verify correlation - Spearman (non-linear)
sns.heatmap(df_pontuation.corr('spearman'), annot=True)

In [ ]:
# Histogram of variables
sns.displot(df_pontuation, x='study_hours')

## Train model

In [ ]:
# Divide training and test
# when there is just one feature, it is necessary to reshape the array
X = df_pontuation.study_hours.values.reshape(-1, 1)
y = df_pontuation.pontuation.values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

In [ ]:
# Instantiate the model to be trained
reg_model = LinearRegression()

In [ ]:
# Train the model
reg_model.fit(X_train, y_train)

In [ ]:
# Print the equation of the model
# y = ax + b
print("The equation of y = {:4f}x + {:4f}".format(reg_model.coef_[0][0], reg_model.intercept_[0]))

## Validate Model - Metrics

In [ ]:
# Predict values based on tests dataset
y_pred = reg_model.predict(X_test)

In [ ]:
# Calculate metric R-squared or Determination Coefficient
# 0 to 1. Represents the percentage of the dependent variable that is explained by the independent variable
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
r2_score(y_test, y_pred)

In [ ]:
# Simple Regression tries to minimize the sum of the squared errors
# Calculate metric Mean Absolute Error (MAE) - difference between the real value and the predicted value
# MAE = Media(y_test - y_pred)
mean_absolute_error(y_test, y_pred)

In [ ]:
# Calculate metric Mean Squared Error (MSE) - difference between the real value and the predicted value
# MSE = Media(y_test - y_pred)^2
mean_squared_error(y_test, y_pred)

In [ ]:
# Calculate metric Root Mean Squared Error (RMSE) - difference between the real value and the predicted value
# MSE = Raiz(Media(y_test - y_pred)^2)
mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
# Graphical analysis of the model
x_axis = range(len(y_test))
plt.figure(figsize=(10, 6))
sns.scatterplot(x=x_axis, y=y_test.reshape(-1), color='blue', label='Real Values')
sns.scatterplot(x=x_axis, y=y_pred.reshape(-1), color='red', label='Predicted Values')
plt.legend()
plt.show()

## Residual Analysis

In [ ]:
# Calculate residues
# difference between the real value and the predicted value
residues = y_test - y_pred

In [ ]:
# Normalize residues (standardization)
# For each set element (X - media) / standard deviation
from scipy.stats import zscore

residues_std = zscore(residues)

In [ ]:
# Verify linearity of model:
  # If residues are between -2 and +2 (in standard deviation), the model is linear
  
# Verify homogeneous variance of model (Homocedasticity):
  # If values are around 0, the model is homocedasticity, otherwise, if it has any pattern, there is heterocedasticity

sns.scatterplot(x=y_pred.reshape(-1), y=residues_std.reshape(-1))
plt.axhline(y=0, color='red')

In [ ]:
# Check if residues follow a normal distribution
# QQ (Quantile-Quantile) Plot
import pingouin as pg

pg.qqplot(residues_std, dist='norm', confidence=0.95)
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Residues in Standard Scale')
plt.show()

In [ ]:
# Normality test - Shapiro-Wilk
# H0: residues follow a normal distribution
# H1: residues do not follow a normal distribution
# If p-value < 0.05, reject H0, otherwise, do not reject H0
stat_shapiro, p_value = shapiro(residues.reshape(-1))
print("Test stastistics: {} and P-Value: {}".format(stat_shapiro, p_value))

In [ ]:
# Normality test - Kolmogorov-Smirnov
# H0: residues follow a normal distribution
# H1: residues do not follow a normal distribution
# If p-value < 0.05, reject H0, otherwise, do not reject H0
stat_ks, p_value_ks = kstest(residues.reshape(-1), 'norm')
print("Test stastistics: {} and P-Value: {}".format(stat_ks, p_value_ks))

# Making Predictions

In [ ]:
reg_model.predict([[30.4]])1

In [ ]:
# predicting reverse - based on pontuation, predict study hours
# y = ax + b
# y - b = ax
# (y - b) / a = x
# x = (y - b) / a
(600 - reg_model.intercept_[0]) / reg_model.coef_[0][0]